In [378]:
import psycopg2
import random
import faker
import pytz
from datetime import datetime, timedelta
from pytz import timezone

# Setup Faker for generating mock data
fake = faker.Faker()

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname="DB_Final_Project", 
    user="postgres", 
    password="bear123321a", 
    host="localhost"
)
cur = conn.cursor()

In [334]:
# Data stores for unique values
users = {}
user_devices = {}
user_friends = {}
user_game_types = {}
game_types = {}
add_fund_record = {}
buy_item_cancel = {}
buy_item = {}
cart = {}
games = {}
game_items = {}
game_game_type = {}
game_reviews = {}
publishers = {}
developers = {}
game_developers = {}
user_games = {}
user_game_statistics = {}
user_inventory = {}
achievements = {}
user_achievements = {}

In [335]:
def generate_users(num=10):
    for _ in range(num):
        user_id = fake.unique.random_number(digits=10)
        users[user_id] = {
            "user_id": user_id,
            "password_hashed": fake.password(length=20),
            "user_name": fake.user_name()[:10],
            "user_description": fake.text(max_nb_chars=300),
            "profile_pic": fake.image_url(),
            "profile_background": fake.image_url(),
            "birth_day": fake.date_of_birth(minimum_age=18, maximum_age=80),
            "email": fake.email(),
            "country": fake.country()[:20],
            "language": fake.language_name()[:20],
            "fund": random.randint(100, 1000),
            "filtering": random.choice([True, False]),
            "notification": random.choice([True, False]),
            "cookies": random.choice([True, False]),
        }

        # Use the correct dictionary values
        cur.execute(
            """
            INSERT INTO public."user" 
            (user_id, password_hashed, user_name, user_description, profile_pic, 
            profile_background, birthday, email, country, language, fund, 
            filtering, notification, cookies) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (
                user_id, 
                users[user_id]['password_hashed'], 
                users[user_id]['user_name'], 
                users[user_id]['user_description'], 
                users[user_id]['profile_pic'], 
                users[user_id]['profile_background'], 
                users[user_id]['birth_day'], 
                users[user_id]['email'], 
                users[user_id]['country'], 
                users[user_id]['language'], 
                users[user_id]['fund'], 
                users[user_id]['filtering'], 
                users[user_id]['notification'], 
                users[user_id]['cookies']
            )
        )
    conn.commit()
generate_users(5)


In [336]:
# Define a function to generate user devices
def generate_user_devices(num=10):
    for _ in range(num):
        user_id = random.choice(list(users.keys()))  # Pick a random user
        device_id = fake.unique.random_number(digits=1)  # Generate a unique device ID
        device_name = fake.word()[:20]  # Generate a device name
        type = fake.word()[:10]  # Generate a device type

        # Insert the device into the database
        cur.execute(
            """
            INSERT INTO public."user_devices" 
            (user_id, device_id, device_name, type) 
            VALUES (%s, %s, %s, %s)
            """,
            (user_id, device_id, device_name, type)
        )
    conn.commit()
generate_user_devices(5)

In [337]:
# Define a function to generate user friends
def generate_user_friends(num=1):
    for _ in range(num):
        user_id = random.choice(list(users.keys()))  # Pick a random user
        friend_id = random.choice(list(users.keys()))  # Pick a random friend

        # Ensure the friend_id is not the same as user_id
        while friend_id == user_id:
            friend_id = random.choice(list(users.keys()))
        # Insert the friendship into the database
        # Insert both directions of the friendship (user_id -> friend_id and friend_id -> user_id)
        cur.execute(
            """
            INSERT INTO public."user_friends" (user_id, friend_id) 
            VALUES (%s, %s)
            """,
            (user_id, friend_id)
        )

        cur.execute(
            """
            INSERT INTO public."user_friends" (user_id, friend_id) 
            VALUES (%s, %s)
            """,
            (friend_id, user_id)
        )
    conn.commit()
generate_user_friends(5)

UniqueViolation: duplicate key value violates unique constraint "user_friends_pkey"
DETAIL:  Key (user_id, friend_id)=(1578659470, 8733850776) already exists.


In [340]:
# Define a function to generate game types
def generate_game_types(num=5):
    for _ in range(num):
        game_type_id = fake.unique.random_number(digits=10)  # Generate a unique game type ID
        game_type_name = fake.word()[:10]  # Generate a random game type name
        
        # Store the generated data (optional, depending on your needs)
        game_types[game_type_id] = {
            "game_type_id": game_type_id,
            "game_type_name": game_type_name,
        }
        
        # Insert the game type into the database
        cur.execute(
            """
            INSERT INTO public."game_types" (game_type_id, game_type_name) 
            VALUES (%s, %s)
            """,
            (game_type_id, game_type_name)
        )
    conn.commit()
generate_game_types(3)

In [341]:
# Define a function to generate user game types
def generate_user_game_types(num=5):
    for _ in range(num):
        user_id = random.choice(list(users.keys()))  # Pick a random user
        game_type_id = random.choice(list(game_types.keys()))  # Generate a unique game type ID
        
        # Store the generated data (optional, depending on your needs)
        user_game_types[(user_id, game_type_id)] = {
            "user_id": user_id,
            "game_type_id": game_type_id,
        }
        
        # Insert the user-game type relationship into the database
        cur.execute(
            """
            INSERT INTO public."user_game_types" (user_id, game_type_id) 
            VALUES (%s, %s)
            """,
            (user_id, game_type_id)
        )
    conn.commit()
generate_user_game_types(3)

In [342]:
# Define a function to generate add fund records
def generate_add_fund_record(num=10):
    # Set a timezone (e.g., UTC) to convert the timestamp
    tz = timezone('UTC')
    
    for _ in range(num):
        add_fund_record_id = fake.unique.random_number(digits=10)  # Unique record ID
        user_id = random.choice(list(users.keys()))  # Select a random user
        fund_change = random.randint(10, 1000)  # Random fund change between 10 and 1000
        
        # Generate timestamp and convert to include time zone
        timestamp = fake.date_time_this_year()  # Random timestamp within this year
        timestamp = tz.localize(timestamp)  # Localize the timestamp to the UTC time zone
        
        # Insert the generated record into the add_fund_record table
        cur.execute(
            """
            INSERT INTO public."add_fund_record" (add_fund_record_id, user_id, fund_change, timestamp)
            VALUES (%s, %s, %s, %s)
            """,
            (add_fund_record_id, user_id, fund_change, timestamp)
        )
    conn.commit()

# Call the function to generate and insert 10 add fund records
generate_add_fund_record(10)

In [343]:
# Define a function to generate games
def generate_games(num=10):
    for _ in range(num):
        # Generate a unique game_id
        game_id = fake.unique.random_number(digits=10)
        
        # Generate random game attributes
        game_name = fake.word()[:20]
        game_description = fake.sentence()[:100]
        system_reuirements = fake.sentence()[:100]
        
        # Store the generated game record in the dictionary (optional)
        games[game_id] = {
            "game_id": game_id,
            "game_name": game_name,
            "game_description": game_description,
            "system_requirements": system_reuirements,
        }
        
        # Insert the generated record into the database
        cur.execute(
            """
            INSERT INTO public."game" (game_id, game_name, game_description, system_reuirements)
            VALUES (%s, %s, %s, %s)
            """,
            (game_id, game_name, game_description, system_reuirements)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 10 game records
generate_games(10)


In [344]:
# Define a function to generate game items
def generate_game_items(num=10):
    if not games:
        raise ValueError("Games have not been generated yet.")
    
    for _ in range(num):
        # Generate a unique item_id
        game_item = fake.unique.random_number(digits=10)
        
        # Pick a random game_id from the existing games
        game_id = random.choice(list(games.keys()))
        
        # Generate random attributes for the game item
        original_price = random.randint(10, 100)
        current_price = random.randint(10, 100)
        special_offer = round(random.uniform(0, 1), 2)
        release_date = fake.date_this_decade()
        
        # Store the generated game item in the dictionary using a tuple (game_item, game_id) as the key
        game_items[(game_item, game_id)] = {
            "item_id": game_item,
            "game_id": game_id,
            "original_price": original_price,
            "current_price": current_price,
            "special_offer": special_offer,
            "release_date": release_date,
        }
        
        # Insert the generated game item into the database
        cur.execute(
            """
            INSERT INTO public."game_item" (item_id, game_id, original_price, current_price, special_offer, release_date) 
            VALUES (%s, %s, %s, %s, %s, %s)
            """,
            (game_item, game_id, original_price, current_price, special_offer, release_date)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 10 game items
generate_game_items(10)


In [345]:
# Define a function to generate buy item records
def generate_buy_item(num=10):
    tz = timezone('UTC')  # Set the timezone (e.g., UTC)
    
    for _ in range(num):
        # Generate a unique buy_item_id
        buy_item_id = fake.unique.random_number(digits=10)
        
        # Pick random user, game, and item from the existing records
        user_id = random.choice(list(users.keys()))
        
        # Pick a (game_id, item_id) tuple from the existing records
        item_id, game_id = random.choice(list(game_items.keys()))
        
        # Random price between 10 and 100
        price = random.randint(10, 100)
        
        # Generate timestamp and localize it to UTC timezone
        timestamp = fake.date_time_this_year()
        timestamp = tz.localize(timestamp)  # Localize to the UTC time zone
        
        # Randomly choose whether the buy item is cancelled
        isCancelled = random.choice([True, False])
        
        # Store the generated buy item record in the dictionary (optional)
        buy_item[(buy_item_id, game_id, item_id)] = {
            "buy_item_id": buy_item_id,
            "user_id": user_id,
            "game_id": game_id,
            "item_id": item_id,
            "price": price,
            "timestamp": timestamp,
            "isCancelled": isCancelled
        }
        
        # Insert the generated record into the database
        cur.execute(
            """
            INSERT INTO public."buy_item" (buy_item_id, user_id, game_id, item_id, price, timestamp, "isCancelled")
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """,
            (buy_item_id, user_id, game_id, item_id, price, timestamp, isCancelled)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 10 buy item records
generate_buy_item(10)

In [346]:
# Define a function to generate buy item cancel records
def generate_buy_item_cancel(num=10):
    if not buy_item:
        raise ValueError("Buy item records have not been generated yet.")
    
    tz = timezone('UTC')  # Set the timezone (e.g., UTC)
    
    # Filter buy items with isCancelled = True
    cancelled_buy_items = {k: v for k, v in buy_item.items() if v["isCancelled"] is True}
    
    if not cancelled_buy_items:
        raise ValueError("No cancelled buy items found.")

    for _ in range(num):
        # Pick a random buy_item_id, game_id, and item_id from the cancelled items
        buy_item_id, game_id, item_id = random.choice(list(cancelled_buy_items.keys()))
        
        # Generate a cancellation timestamp and localize it to UTC timezone
        timestamp = fake.date_time_this_year()
        timestamp = tz.localize(timestamp)  # Localize to UTC time zone
        
        # Store the generated cancel record in the dictionary (optional)
        buy_item_cancel[(buy_item_id, game_id, item_id)] = {
            "buy_item_id": buy_item_id,
            "game_id": game_id,
            "item_id": item_id,
            "timestamp": timestamp
        }
        
        # Insert the cancel record into the database
        cur.execute(
            """
            INSERT INTO public."buy_item_cancel" (buy_item_id, game_id, item_id, timestamp)
            VALUES (%s, %s, %s, %s)
            """,
            (buy_item_id, game_id, item_id, timestamp)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 5 buy item cancel records
generate_buy_item_cancel(2)

In [236]:
# Define a function to generate cart items
def generate_cart(num=10):
    if not games or not game_items:
        raise ValueError("Games or game items have not been generated yet.")
    
    for _ in range(num):
        # Randomly pick a user, game, and item
        user_id = random.choice(list(users.keys()))
        # Pick a (game_id, item_id) tuple from the existing records
        item_id, game_id = random.choice(list(game_items.keys()))
        
        # Store the generated cart item in the dictionary (optional)
        cart[(user_id, game_id, item_id)] = {
            "user_id": user_id,
            "game_id": game_id,
            "item_id": item_id
        }
        
        # Insert the cart item into the database
        cur.execute(
            """
            INSERT INTO public."cart" (user_id, game_id, item_id)
            VALUES (%s, %s, %s)
            """,
            (user_id, game_id, item_id)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 10 cart items
generate_cart(10)


In [237]:
# Define a function to generate game game types
def generate_game_game_type(num=5):
    if not games or not game_types:
        raise ValueError("Games or game types have not been generated yet.")
    
    for _ in range(num):
        # Select a random game_id and game_type_id
        game_id = random.choice(list(games.keys()))
        game_type_id = random.choice(list(game_types.keys()))
        
        # Store the generated game-game type pair (optional)
        game_game_type[(game_id, game_type_id)] = {
            "game_id": game_id,
            "game_type_id": game_type_id
        }
        
        # Insert the generated pair into the database
        cur.execute(
            """
            INSERT INTO public."game_game_type" (game_id, game_type_id)
            VALUES (%s, %s)
            """,
            (game_id, game_type_id)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 5 game-game type records
generate_game_game_type(5)

In [238]:
# Define a function to generate game reviews
def generate_game_reviews(num=10):
    if not games or not users:
        raise ValueError("Games or users have not been generated yet.")
    
    for _ in range(num):
        # Generate a unique review_id
        review_id = fake.unique.random_number(digits=10)
        
        # Pick random game_id and user_id from the existing records
        game_id = random.choice(list(games.keys()))
        user_id = random.choice(list(users.keys()))
        
        # Generate review attributes
        review_timestamp = fake.date_time_this_year()
        rating = random.randint(1, 5)
        text = fake.sentence()
        
        # Store the generated review record (optional)
        game_reviews[review_id] = {
            "review_id": review_id,
            "game_id": game_id,
            "user_id": user_id,
            "review_timestamp": review_timestamp,
            "rating": rating,
            "text": text
        }
        
        # Insert the review record into the database
        cur.execute(
            """
            INSERT INTO public."game_reviews" (review_id, game_id, user_id, review_timestamp, rating, text)
            VALUES (%s, %s, %s, %s, %s, %s)
            """,
            (review_id, game_id, user_id, review_timestamp, rating, text)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 10 game review records
generate_game_reviews(10)

In [348]:
# Define a function to generate publishers
def generate_publishers(num=10):
    for _ in range(num):
        # Generate unique publisher ID and other attributes
        publisher_id = fake.unique.random_number(digits=10)
        publisher_name = fake.company()[:20]
        description = fake.sentence()[:100]
        
        # Store the generated publisher record (optional)
        publishers[publisher_id] = {
            "publisher_id": publisher_id,
            "publisher_name": publisher_name,
            "description": description,
        }
        
        # Insert the publisher data into the database
        cur.execute(
            """
            INSERT INTO public."publishers" (publisher_id, publisher_name, description)
            VALUES (%s, %s, %s)
            """,
            (publisher_id, publisher_name, description)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 10 publisher records
generate_publishers(10)

In [349]:
# Define a function to generate developers
def generate_developers(num=10):
    if not publishers:
        raise ValueError("Publishers have not been generated yet.")
    
    for _ in range(num):
        # Generate unique developer ID and other attributes
        developer_id = fake.unique.random_number(digits=10)
        developer_name = fake.company()[:20]
        description = fake.sentence()[:100]
        publisher_id = random.choice(list(publishers.keys()))  # Randomly select a publisher
        
        # Store the generated developer record (optional)
        developers[developer_id] = {
            "developer_id": developer_id,
            "developer_name": developer_name,
            "description": description,
            "publisher_id": publisher_id,
        }
        
        # Insert the developer data into the database
        cur.execute(
            """
            INSERT INTO public."developers" (developer_id, developer_name, description, publisher_id)
            VALUES (%s, %s, %s, %s)
            """,
            (developer_id, developer_name, description, publisher_id)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 10 developer records
generate_developers(10)


In [350]:
# Define a function to generate game developers
def generate_game_developers(num=10):
    if not developers or not games:
        raise ValueError("Developers or games have not been generated yet.")
    
    for _ in range(num):
        # Pick a random developer and game
        developer_id = random.choice(list(developers.keys()))
        game_id = random.choice(list(games.keys()))
        
        # Store the game-developer relationship in the dictionary (optional)
        game_developers[(developer_id, game_id)] = {
            "developer_id": developer_id,
            "game_id": game_id
        }
        
        # Insert the game-developer relationship into the database
        cur.execute(
            """
            INSERT INTO public."game_developers" (developer_id, game_id)
            VALUES (%s, %s)
            """,
            (developer_id, game_id)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 10 game developer relationships
generate_game_developers(10)

UniqueViolation: duplicate key value violates unique constraint "GAME_DEVELOPERS_pkey"
DETAIL:  Key (developer_id, game_id)=(2676773150, 1109147998) already exists.


In [353]:
# Define a function to generate user games
def generate_user_games(num=10):
    if not games:
        raise ValueError("Games have not been generated yet.")
    
    for _ in range(num):
        # Pick a random user and game
        user_id = random.choice(list(users.keys()))
        game_id = random.choice(list(games.keys()))
        
        # Generate installation date
        installed_date = fake.date_this_decade()
        
        # Generate uninstallation date, or set it to None if not uninstalled
        uninstalled_date = fake.date_this_decade() if random.choice([True, False]) else None
        
        # Store the user-game relationship in the dictionary (optional)
        user_games[(user_id, game_id)] = {
            "user_id": user_id,
            "game_id": game_id,
            "installed_date": installed_date,
            "uninstalled_date": uninstalled_date
        }
        
        # Insert the user-game relationship into the database
        cur.execute(
            """
            INSERT INTO public."user_games" (user_id, game_id, installed_date, uninstalled_date)
            VALUES (%s, %s, %s, %s)
            """,
            (user_id, game_id, installed_date, uninstalled_date)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 10 user-game relationships
generate_user_games(10)

In [354]:
# Define a function to generate user game statistics
def generate_user_game_statistics(num=10):
    if not user_games:  # Ensure that user_games are generated
        raise ValueError("User games have not been generated yet.")
    
    for _ in range(num):
        # Pick a random (user_id, game_id) pair from user_games
        user_id, game_id = random.choice(list(user_games.keys()))
        
        # Generate random statistics for played time (in minutes) and number of achievements
        played_time = timedelta(minutes=random.randint(1, 1000))
        achievement_num = random.randint(0, 50)
        
        # Store the user game statistics in the dictionary (optional)
        user_game_statistics[(user_id, game_id)] = {
            "user_id": user_id,
            "game_id": game_id,
            "played_time": played_time,
            "achievement_num": achievement_num
        }
        
        # Insert the generated user game statistics into the database
        cur.execute(
            """
            INSERT INTO public."user_game_statistics" (user_id, game_id, played_time, achievement_num)
            VALUES (%s, %s, %s, %s)
            """,
            (user_id, game_id, played_time, achievement_num)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 10 user game statistics
generate_user_game_statistics(2)

In [355]:
# Define a function to generate user inventory
def generate_user_inventory(num=10):
    if not games or not game_items or not user_games:
        raise ValueError("Games, game items, or user games have not been generated yet.")
    
    # Define a timezone (you can replace 'UTC' with any other timezone as needed)
    tz = pytz.timezone('UTC')
    
    for _ in range(num):
        # Pick a random (user_id, game_id) from the user_games dictionary
        user_id, game_id = random.choice(list(user_games.keys()))
        
        # Filter the game_items to those that belong to the selected game_id
        available_items = [item_id for item_id, item in game_items.items() if item['game_id'] == game_id]
        
        if not available_items:
            continue  # Skip if no items are available for the selected game_id
        
        # Pick a random item_id from the available items for the selected game_id
        item_id, game_id = random.choice(available_items)
        
        # Generate random dates for acquisition and "not owned" status
        acquired_date = fake.date_this_century()  # Date when the item was acquired
        # Convert acquired_date (a datetime.date object) to datetime.datetime and localize
        acquired_date = datetime.combine(acquired_date, datetime.min.time())  # Add time part to make it a datetime
        acquired_date = tz.localize(acquired_date)  # Make acquired_date timezone-aware
        
        not_owned_date = fake.date_this_century() if random.choice([True, False]) else None
        if not_owned_date:
            # Convert not_owned_date (a datetime.date object) to datetime.datetime and localize
            not_owned_date = datetime.combine(not_owned_date, datetime.min.time())  # Add time part
            not_owned_date = tz.localize(not_owned_date)  # Make not_owned_date timezone-aware
        
        # If not_owned_date is set, ensure it happens after the acquired_date
        if not_owned_date and not_owned_date < acquired_date:
            not_owned_date = fake.date_between(start_date=acquired_date)  # Corrected to use `date_between`
            # Convert new not_owned_date to datetime and localize
            not_owned_date = datetime.combine(not_owned_date, datetime.min.time())
            not_owned_date = tz.localize(not_owned_date)
        
        # Store the user inventory record
        user_inventory[(user_id, game_id, item_id)] = {
            "user_id": str(user_id),  # Ensure user_id is a string and truncated to 10 characters
            "game_id": str(game_id),  # Ensure game_id is a string and truncated to 10 characters
            "item_id": str(item_id),  # Ensure item_id is a string and truncated to 10 characters
            "acquired_date": acquired_date,
            "not_owned_date": not_owned_date
        }
        # Insert the user inventory record into the database
        cur.execute(
            """
            INSERT INTO public."user_inventory" (user_id, game_id, item_id, acquired_date, not_owned_date)
            VALUES (%s, %s, %s, %s, %s)
            """,
            (str(user_id), str(game_id), str(item_id), acquired_date, not_owned_date)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 10 user inventory records
generate_user_inventory(10)


In [356]:
# Define a function to generate achievements
def generate_achievements(num=10):
    if not games:
        raise ValueError("Games have not been generated yet.")
    
    for _ in range(num):
        # Generate a unique achievement_id
        achievement_id = fake.unique.random_number(digits=10)
        
        # Pick a random game_id from the existing games
        game_id = random.choice(list(games.keys()))
        
        # Generate random attributes for the achievement
        achievement_name = fake.word()[:10]
        achievement_description = fake.sentence()[:100]
        
        # Store the generated achievement in the dictionary
        achievements[(achievement_id, game_id)] = {
            "achievement_id": achievement_id,
            "game_id": game_id,
            "achievement_name": achievement_name,
            "achievement_description": achievement_description,
        }
        
        # Insert the generated achievement into the database
        cur.execute(
            """
            INSERT INTO public."achievements" (achievement_id, game_id, achievement_name, achievement_description) 
            VALUES (%s, %s, %s, %s)
            """,
            (achievement_id, game_id, achievement_name, achievement_description)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 10 achievements
generate_achievements(10)

In [367]:
# Define a function to generate user achievements
def generate_user_achievements(num=10):
    if not achievements:
        raise ValueError("Achievements have not been generated yet.")
    
    tz = pytz.timezone('UTC')  # Set your desired timezone (e.g., 'UTC')
    
    for _ in range(num):
        # Pick a random (user_id, game_id) from the user_games dictionary
        user_id, game_id = random.choice(list(user_games.keys()))
        
        # Pick a random achievement_id from the existing achievements for the selected game_id
        available_achievements = [ach_id for ach_id, ach in achievements.items() if ach['game_id'] == game_id]

        if not available_achievements:
            continue  # Skip if no achievements are available for the selected game_id
        
        # Pick a random achievement_id from the available achievements for the selected game_id
        achievement_id, game_id = random.choice(available_achievements)
        
        # Generate a random date for when the achievement was achieved
        achieved_date = fake.date_this_century()
        
        # Convert the date to a datetime object and make it timezone-aware
        achieved_date = datetime.combine(achieved_date, datetime.min.time())  # Convert to datetime
        achieved_date = tz.localize(achieved_date)  # Make it timezone-aware
        
        # Store the generated user achievement in the dictionary
        user_achievements[(user_id, game_id, achievement_id)] = {
            "user_id": user_id,
            "game_id": game_id,
            "achievement_id": achievement_id,
            "achieved_date": achieved_date
        }
        
        # Insert the generated user achievement into the database
        cur.execute(
            """
            INSERT INTO public."user_achievements" (user_id, game_id, achievement_id, achieved_date)
            VALUES (%s, %s, %s, %s)
            """,
            (user_id, game_id, achievement_id, achieved_date)
        )
    
    # Commit the changes to the database
    conn.commit()

# Call the function to generate and insert 10 user achievements
generate_user_achievements(10)

UniqueViolation: duplicate key value violates unique constraint "USER_ACHIEVEMENTS_pkey"
DETAIL:  Key (user_id, achievement_id)=(3369644877, 5072191435) already exists.


In [389]:
import psycopg2
import random
import faker
import pytz
from datetime import datetime, timedelta
from pytz import timezone

# Setup Faker for generating mock data
fake = faker.Faker()

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname="DB_Final_Project", 
    user="postgres", 
    password="bear123321a", 
    host="localhost"
)
cur = conn.cursor()

# List of SQL delete statements
sql_delete_queries = [
    "DELETE FROM public.\"user_achievements\";",
    "DELETE FROM public.\"user_inventory\";",
    "DELETE FROM public.\"user_game_statistics\";",
    "DELETE FROM public.\"user_games\";",
    "DELETE FROM public.\"game_developers\";",
    "DELETE FROM public.\"developers\";",
    "DELETE FROM public.\"publishers\";",
    "DELETE FROM public.\"game_reviews\";",
    "DELETE FROM public.\"game_game_type\";",
    "DELETE FROM public.\"game_item\";",
    "DELETE FROM public.\"games\";",
    "DELETE FROM public.\"cart\";",
    "DELETE FROM public.\"buy_item\";",
    "DELETE FROM public.\"buy_item_cancel\";",
    "DELETE FROM public.\"add_fund_record\";",
    "DELETE FROM public.\"user_friends\";",
    "DELETE FROM public.\"user_devices\";",
    "DELETE FROM public.\"user_game_types\";",
    "DELETE FROM public.\"game_types\";"
]

# Execute each SQL query
for query in sql_delete_queries:
    cur.execute(query)

# Commit the changes to the database
conn.commit()

ForeignKeyViolation: update or delete on table "game_item" violates foreign key constraint "buy_item_game_id_item_id_fkey" on table "buy_item"
DETAIL:  Key (game_id, item_id)=(6347982239, 8021334440) is still referenced from table "buy_item".
